In [ ]:
import csv

sum_pubs = 0

collected_venues = {}
################ Discard Venues with less than 10 publications and merge venues over years
with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Publication_venue_stats.csv", 'r') as file:
  csvreader = csv.reader(file)
  first = 0

  for row in csvreader:
    if first < 1:
      first += 1
      continue

    sum_pubs += int(row[1])

    # Merge venues which repeat over the years
    venue = ''.join([i for i in row[0] if not i.isdigit()])

    if venue not in collected_venues:
      collected_venues[venue] = int(row[1])
    else:
      collected_venues[venue] += int(row[1])

print('Verifying final amount of publications studied are ', sum_pubs)
#Verifying final amount of publications studied are  18592 --> Inquire from Zainab about the missing 5 papers

print('#Venues in Total ', len(collected_venues))

Verifying final amount of publications studied are  18592
#Venues in Total  4400


In [ ]:
final_collected_venues = []
flagged_venues = []
for venue, npapers in collected_venues.items():
    if npapers < 10:
      #if "image" in venue.lower() or "computer vision" in venue.lower():
      #  print("Skipped ", venue)
      flagged_venues.append(venue)
      continue

    final_collected_venues.append(venue)

print('#Venues after removing outliers in Total ', len(final_collected_venues))

with open('flagged_venues.txt', 'w') as f:
  for venue in flagged_venues:
    f.write(venue+'\n')


#Venues after removing outliers in Total  334


In [ ]:
################ Keep safe phrases and safe venues, and flag the rest to be checked manually
safe_phrases = ["intelligence systems", "intelligent systems", "computational vision", "visual", "image", "imaging", "computer vision", "pattern recognition", "signal processing", "machine learning", "healthcare", "health", "health care", "medical",
                "remote sensing", "robotic", "multimedia", "medicine", "data analysis", "big data", "biomed", "applications"]

safe_venues = []
needs_review_venues = []
for query_venue in final_collected_venues:
    flag = True
    for phrase in safe_phrases:
      if phrase in query_venue.lower():
        flag = False
        break

    if flag:
      needs_review_venues.append(query_venue)
    else:
      safe_venues.append(query_venue)

print('Number of Publication Venues to be Reviewed', len(needs_review_venues))
with open('needs_review_venues.txt', 'w') as f:
  for venue in needs_review_venues:
    f.write(venue + '\n')

with open('safe_venues.txt', 'w') as f:
  for venue in safe_venues:
    f.write(venue + '\n')
# Discard All publications from publication venues with less than 5 publications (most probably its an outlier)
# Merge venues that are re-occurring over the years
# Check manually the rest of publication venues/journals and discard the ones with nouns or noun phrases that are mostly related to computer vision or a common application of it
# The flagged venues can be used to check 5 random papers from it to confirm that it's relevant to computer vision

Number of Publication Venues to be Reviewed 196


In [ ]:
###### Load Needs to be reviewed venues after manual filtration
import csv

needs_review_venues = []
with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Flagged_Venues_Journals-Needs_Review_Final.csv", 'r') as file:
  csvreader = csv.reader(file)

  first = 0
  for row in csvreader:
    if first < 1:
      first += 1
      continue

    needs_review_venues.append(row[0])

print(needs_review_venues)

['Lecture Notes in Computer Science (including subseries Lecture Notes in Artificial Intelligence and Lecture Notes in Bioinformatics)', 'Lecture Notes in Networks and Systems', 'Communications in Computer and Information Science', 'Computers, Materials and Continua', 'National Radio Science Conference, NRSC, Proceedings', 'PLoS ONE', 'CEUR Workshop Proceedings', 'Electronics (Switzerland)', 'IEEE AFRICON Conference', 'Monthly Notices of the Royal Astronomical Society', 'Proceedings of the International Conference on Microelectronics, ICM', 'Diagnostics', 'Journal of Ambient Intelligence and Humanized Computing', 'Geocarto International', 'Journal of Physics: Conference Series', 'Wireless Communications and Mobile Computing', 'Sustainability (Switzerland)', 'Astronomy and Astrophysics', 'International Journal of Applied Earth Observation and Geoinformation', 'Arabian Journal of Geosciences', 'Symmetry', 'Heliyon', 'Advances in Space Research', 'IEEE International Symposium on Industria

In [ ]:
######## Remove Flagged publications
import csv

with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Collection/Scopus_Result.csv", 'r') as file:
  csvreader = csv.reader(file)

  with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Scopus_Result_Filtered.csv", 'w') as file2:
    first = 0
    csvwriter = csv.writer(file2)
    for row in csvreader:
      if first < 1:
        first += 1
        csvwriter.writerow(row)
        continue

      pub_venue = ''.join([i for i in row[18] if not i.isdigit()])
      if pub_venue not in flagged_venues:
        csvwriter.writerow(row)


In [ ]:
######## Randomly sample publications to be reviewed in a priority based sampling
import random

needs_review_pubs = {}
with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Scopus_Result_Filtered.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    if first < 1:
      first += 1
      continue

    pub_venue = ''.join([i for i in row[18] if not i.isdigit()])
    if pub_venue in needs_review_venues:
      if pub_venue not in needs_review_pubs:
        needs_review_pubs[pub_venue] = []
      needs_review_pubs[pub_venue].append(row)

total_pubs_not_flagged = 0
venues_sorted = {}
for venue, pubs in needs_review_pubs.items():
  venues_sorted[venue] = len(pubs)
  total_pubs_not_flagged += len(pubs)

print('Total publications that needs review', total_pubs_not_flagged)
for venue, _ in venues_sorted.items():
  venues_sorted[venue] /= total_pubs_not_flagged
print('Priorities: ', venues_sorted)
max_prob = max(venues_sorted.values())
min_prob = min(venues_sorted.values())

# We're not using these probabilities directly to sample with cause we need to ensure we only can have max 10 papers per venue for practical reasons
MAX_PAPERS = 10
MIN_PAPERS = 2
selected_pubs = {}
for venue, pubs in needs_review_pubs.items():
  number_papers =  round( ((venues_sorted[venue] - min_prob) / (max_prob - min_prob)) * (MAX_PAPERS - MIN_PAPERS) + MIN_PAPERS)
  selected_pubs[venue] = random.sample(pubs, number_papers)


with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Scopus_Result_Needs_Review.csv", 'w') as file2:
    csvwriter = csv.writer(file2)

    for venue, pubs in selected_pubs.items():
      for pub in pubs:
        csvwriter.writerow(pub)

Total publications that needs review 2581
Priorities:  {'Wireless Communications and Mobile Computing': 0.01084850833010461, 'IEEE AFRICON Conference': 0.015497869043006587, 'Expert Systems': 0.0038744672607516468, 'Concurrency and Computation: Practice and Experience': 0.006199147617202635, 'Computers, Materials and Continua': 0.05579232855482371, 'Lecture Notes in Computer Science (including subseries Lecture Notes in Artificial Intelligence and Lecture Notes in Bioinformatics)': 0.2413793103448276, 'PLoS ONE': 0.02479659046881054, 'Communications in Computer and Information Science': 0.05734211545912437, 'IET Biometrics': 0.004261913986826811, 'ICEIS  - Proceedings of the th International Conference on Enterprise Information Systems': 0.0038744672607516468, 'Monthly Notices of the Royal Astronomical Society': 0.014722975590856257, 'SAIEE Africa Research Journal': 0.005424254165052305, 'Lecture Notes in Networks and Systems': 0.063541263076327, 'CEUR Workshop Proceedings': 0.02479659

In [ ]:
# Use manually flagged publications from the above randomly sampled ones to finalize the flagging of the venues itself

import csv

venues_pubs = {}

first = 0
with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Scopus_Result_Filtered_By_Topic.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    if first < 1:
      first += 1
      continue

    pub_venue = ''.join([i for i in row[20] if not i.isdigit()])

    if pub_venue not in venues_pubs:
      venues_pubs[pub_venue] = []

    venues_pubs[pub_venue].append(row)

reviewed_and_flagged_venues = []
for venue, pubs_list in venues_pubs.items():
  majority_vote = 0
  for pub in pubs_list:
    if pub[1] == '1':
      majority_vote += 1

  if majority_vote >= len(pubs_list)//2:
    reviewed_and_flagged_venues.append(venue)

print('Reviewed and Flagged Venues: #', len(reviewed_and_flagged_venues))
with open("reviewed_and_flagged.csv", 'w') as f:
  for venue in reviewed_and_flagged_venues:
    f.write(venue+'\n')

Reviewed and Flagged Venues: # 16


In [ ]:
venues = []

first = 0
with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Final_Flagged/Final_Auto_Flagged.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    if first < 1:
      first += 1
      continue
    if len(row) == 0:
      continue

    venues.append(row[0])

first = 0
with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Final_Flagged/Final_Reviewed_and_Flagged.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    if first < 1:
      first += 1
      continue

    if len(row) == 0:
      continue

    venues.append(row[0])

print('Total Flagged Venues #', len(venues))

first = 0
filtered_publications = []
with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Verification/Scopus_Result_Filtered_By_Topic.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    if first < 1:
      first += 1
      continue
    if row[0] == '1' or row[1] == '1':
      filtered_publications.append(row[6])

first = 0
final_publications = []
with open("/content/drive/MyDrive/Ro'ya CV4Africa Community Files/Bibliometric Study/Work_Phases/Data_Collection/Scopus_Result.csv", 'r') as file:
  csvreader = csv.reader(file)
  for row in csvreader:
    if first < 1:
      first += 1
      continue

    current_venue = ''.join([i for i in row[18] if not i.isdigit()])

    if current_venue in venues:
      continue

    if row[4] in filtered_publications:
      continue

    final_publications.append(row)

with open("Final_Filtered_Publications.csv", "w") as f:
  csvwriter = csv.writer(f)

  for publication in final_publications:
    csvwriter.writerow(publication)

print("total Publications filtered out #", len(final_publications))

Total Flagged Venues # 4036
total Publications filtered out # 12164
